In [4]:
from bs4 import BeautifulSoup
#from tabulate import tabulate
import requests
import pandas as pd

In [43]:
# Set website and user-agent or page will obfuscate datea
cape_website = "http://www.cape.ucsd.edu/responses/Results.aspx?Name=&CourseNumber=cse"
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

In [44]:
# Set up requests BeautifulSoup scraper
request = requests.get(cape_website, headers=headers)
soup = BeautifulSoup(request.content, 'html.parser')

In [71]:
table = soup.find_all('table')[0]


In [77]:
listOfValue = {}
counter = 0
tableList =  table.find_all('tbody')[0]
ClassList = tableList.find_all('tr')

In [78]:

for i in ClassList:
    listOfValue[i.find('a').text] = []

for i in ClassList:
    listOfValue[i.find('a').text].append(i.find_all('td')[2].text)

    

In [79]:
for key, value in listOfValue.items():
    print(key)
    print(value)
    print()

CSE 100 - Advanced Data Structures (A)
['WI18', 'FA17', 'SP17', 'WI17', 'FA16', 'S216', 'SP16', 'WI16', 'FA15', 'SP15', 'WI15', 'FA14', 'SP14', 'WI14', 'FA13', 'S213', 'SP13', 'WI13', 'FA12']

CSE 100 - Advanced Data Structures (B)
['WI18', 'FA17', 'S217', 'SP17', 'WI17', 'FA16', 'SP16', 'WI16', 'FA15', 'SP15', 'WI15', 'FA14', 'S114', 'WI14', 'SP13']

CSE 101 - Design & Analysis of Algorithm (A)
['WI18', 'FA17', 'S217', 'S117', 'SP17', 'WI17', 'FA16', 'S116', 'SP16', 'WI16', 'FA15', 'S115', 'SP15', 'WI15', 'FA14', 'S114', 'SP14', 'WI14', 'FA13', 'S113', 'SP13', 'FA12', 'FA10', 'SU10', 'WI10', 'FA09', 'SU09', 'SP09', 'WI09', 'FA08', 'SU08', 'SP08', 'WI08', 'FA07', 'SU07']

CSE 105 - Theory of Computation (A)
['WI18', 'FA17']

CSE 105 - Theory of Computation (B)
['WI18']

CSE 107 - Intro to Modern Cryptography (A)
['WI18', 'FA17', 'WI17', 'WI16', 'WI15', 'WI14']

CSE 11 - Intr/Computer Sci&Obj-Ori;:Java (A)
['WI18', 'FA17', 'S217', 'S117', 'SP17', 'WI17', 'S216', 'S116', 'SP16', 'WI16', 

In [80]:
import json
with open('DATA.json', 'w') as fp:
    json.dump(listOfValue, fp)